In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import gzip
import pandas as pd

In [2]:
"""Noms de tots els fitxers que s'han d'utilitzar"""

df_names = ['Aura_with_level1', 'Guillem_with_level1', 'Guillem_with_level2', 'Guillem_without_level2', 'Pau_with_level1', 'Fiona_with_level1', 'Oscar_with_level1', 'Salva_with_level1', 'Salva_with_level3','Pau_without_level2', 'Fiona_without_level2', 'Oscar_without_level2', 'Salva_without_level2' ]
df_names_csv = [i+'.csv' for i in df_names]

## Funcions per treure les dades que ens dona l'eyetracker

In [21]:
def get_data(fitxer):

    """Aquesta funció agafa el gazedata i ens retorna la informació d'aquest en format .csv, en aquest cas només retornem timestamp, gaze2d i els diàmetres de les pupiles"""

    file = gzip.open('Data/raw/'+ fitxer + '/gazedata.gz','rb')
    file_content = file.read().decode("utf-8")
    file.close()
    data = []
    left_open = []
    right_open = []
    contents = file_content.split("\n")

    for content in contents[:-1]:
        json_content = json.loads(content)
        if json_content['data'] != {}:
            if json_content['data']['eyeleft'] == {}:
                dades = {
                    'timestamp': json_content['timestamp'],
                    'gaze2d_x': json_content['data']['gaze2d'][0],
                    'gaze2d_y': json_content['data']['gaze2d'][1],
                    'eyeleft_pupildiameter': 0,
                    'eyeright_pupildiameter': json_content['data']['eyeright']['pupildiameter']
                }

            elif json_content['data']['eyeright'] == {}:
                dades = {
                    'timestamp': json_content['timestamp'],
                    'gaze2d_x': json_content['data']['gaze2d'][0],
                    'gaze2d_y': json_content['data']['gaze2d'][1],
                    'eyeleft_pupildiameter': json_content['data']['eyeleft']['pupildiameter'],
                    'eyeright_pupildiameter': 0
                }

            else:
                dades = {
                    'timestamp': json_content['timestamp'],
                    'gaze2d_x': json_content['data']['gaze2d'][0],
                    'gaze2d_y': json_content['data']['gaze2d'][1],
                    'eyeleft_pupildiameter': json_content['data']['eyeleft']['pupildiameter'],
                    'eyeright_pupildiameter': json_content['data']['eyeright']['pupildiameter']
                }
            
        else:
            dades = {
                'timestamp': json_content['timestamp'],
                'gaze2d_x': 0,
                'gaze2d_y': 0,
                'eyeleft_pupildiameter': 0,
                'eyeright_pupildiameter': 0
                }

        data.append(dades)

    data = pd.DataFrame(data)

    data.to_csv('Data/gazedata_csv/'+'gazedata_'+fitxer+'.csv', index=False)

    return data


In [22]:
for i in df_names:
    get_data(i)

## Funcions pels csv que ens torna el Heat the Chair

In [19]:
def fix_csv_heatthechair (fitxer):

    """Amb aquesta funció canviem el format de les dates i també fem que les interrupcions en que el seu begin_time és now (en el csv és 0) ens aparegui com a begin time el moment en que arriba la notificació"""

    df = pd.read_csv('Data/csv_heatthechair/'+fitxer)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df['interruption_begin_time'] = pd.to_datetime(df['interruption_begin_time'], unit='s')
    df['interruption_due_time'] = pd.to_datetime(df['interruption_due_time'], unit='s')

    # Com originalment el format de la data està en unix, el 0 en datetime és el 1970-01-01 00:00:00
    zero_time = pd.to_datetime('1970-01-01 00:00:00')
    df['interruption_begin_time'] = df['interruption_begin_time'].apply(lambda x: 0 if x == zero_time else x)
    df['interruption_due_time'] = df['interruption_due_time'].apply(lambda x: 0 if x == zero_time else x)

    # En aquestes dues llistes guardem els moments en que comencen i acaben les interrupcion que comencen 'now'
    list_interruption_begin_time = []
    list_interruption_due_time = []

    # Per poder canviar elements concrets del df busco els indexs de les columnes on es voldran fer el canvis
    columns = df.columns.to_list()
    loc_interruption_begin_time = columns.index('interruption_begin_time')
    loc_interruption_due_time = columns.index('interruption_due_time')

    for i in range(len(df)):
        if (df.iloc[i]['interruption_begin_time'] == 0) & (df.iloc[i]['interruption_due_time'] != 0):

            # Si la interruption due time no està en list_interruption_due_time vol dir que comença ara la interrupció
            if df.iloc[i]['interruption_due_time'] not in list_interruption_due_time:
                df.iloc[i,loc_interruption_begin_time] = df.iloc[i]['timestamp']
                # Guardem el interruption_due_time per saber que ja ha començat aquesta interrupció 
                list_interruption_due_time.append(df.iloc[i]['interruption_due_time'])
                list_interruption_begin_time.append(df.iloc[i]['interruption_begin_time'])
            else:
                # Si la interrucpió acaba entrarà en aquest else i el interruption_begin_time el posem el temps en què va començar la interrupció, per això s'utilitza la list_interruption_begin_time
                df.iloc[i,loc_interruption_begin_time] = list_interruption_begin_time[list_interruption_due_time.index(df.iloc[i]['interruption_due_time'])]

    df.to_csv('Data/csv_interruptions/'+fitxer, index=False)

    return(df)




In [16]:
for i in df_names_csv:
    fix_csv_heatthechair(i)

/tmp/ipykernel_7436/2200089234.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2023-10-10 15:07:27' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[i,loc_interruption_begin_time] = df.iloc[i]['timestamp']


## COORDINAR ELS TIMESTAMP

El timestamp que ens retornen les ulleres són el temps que ha trascorregut des de que s'ha iniciat la gravació. En l'arxiu recording.g3 tenirm el moment d'inici (creation) i utilitzant-lo podem posar la data exacta de cada mesura.

In [18]:
def coordinar_timestamp(gazedata, interruptions, fitxer):
    file = open('Data/raw/'+fitxer+'/recording.g3','r')       
    file_content = file.read()
    file.close()
    file_content = json.loads(file_content)
    start_recording = pd.Timestamp(file_content['created'])
    start_recording = start_recording.tz_convert(tz=file_content['timezone'])
    new_timestamp = start_recording + pd.to_timedelta(gazedata['timestamp'], unit='s') 

    new_timestamp = new_timestamp.dt.tz_localize(None)
    
    return new_timestamp

In [178]:
# En el cas de gazedata_Aura_with_level1
interruptions = pd.read_csv('Data/csv_interruptions/Aura_with_level1.csv')
gazedata = pd.read_csv('Data/gazedata_csv/gazedata_Aura_with_level1')

gazedata['timestamp'] = coordinar_timestamp(gazedata, interruptions, '20231010T143052Z')
gazedata

,timestamp,gaze2d_x,gaze2d_y,eyeleft_pupildiameter,eyeright_pupildiameter
0,2023-10-10 16:30:52.263946,0.5885,0.8835,3.173,2.834
1,2023-10-10 16:30:52.283932,0.5886,0.8800,3.177,2.837
2,2023-10-10 16:30:52.304018,0.5887,0.8785,3.185,2.842
3,2023-10-10 16:30:52.324004,0.5888,0.8769,3.188,2.845
4,2023-10-10 16:30:52.344091,0.5890,0.8753,3.199,2.849
...,...,...,...,...,...
33801,2023-10-10 16:42:09.649491,0.4604,0.6739,4.144,3.499
33802,2023-10-10 16:42:09.669479,0.4604,0.6729,4.144,3.502
33803,2023-10-10 16:42:09.689563,0.4602,0.6714,4.157,3.499
33804,2023-10-10 16:42:09.709551,0.4628,0.6645,4.152,3.372


In [14]:
def trobar_workload(interruptions):

    """Amb aquesta funció trobarem quin workload hi ha en cada instant que dura el joc"""

    interruption_in = interruptions[interruptions['interruption_in'] != 0]
    interruption_out = interruptions[interruptions['interruption_out'] != 0]
    interruption_in = interruption_in.sort_values(by=['interruption_in'] )
    interruption_out = interruption_out.sort_values(by=['interruption_out'] )
    time_interruption_in = list(pd.to_datetime(interruption_in['timestamp']).dt.strftime("%H:%M:%S"))
    time_interruption_out = list(pd.to_datetime(interruption_out['timestamp']).dt.strftime("%H:%M:%S"))

    end_time = interruptions['timestamp'].iloc[-1][-8:]

    # Quan s'acabi la partida hi ha haurà interrupcions que encara seguiran però no s'hauran resolt, això fa qut time_interrution_in i time_interruption_out tinguin llargades diferents
    # per això per cada interrupció que no ha acabat posarem que el temps en que acaba és quan acaba el joc (end_time)
    diferencia = len(time_interruption_in) - len(time_interruption_out)
    for i in range(0,diferencia):
        time_interruption_out.append(end_time)

    # Per poder comparar temps passo el temps de time_interruption_ a timedelta
    time_interruption_in_timedelta = list()
    time_interruption_out_timedelta = list()
    for i in time_interruption_in:
        time_interruption_in_timedelta.append(pd.to_timedelta(i))
    for i in time_interruption_out:
        time_interruption_out_timedelta.append(pd.to_timedelta(i))

    # Creem una seqüència amb tots els segons que dura el joc
    # timedelta_range no accepta com arguments Timestamp, per això interruptions['timestamp'] no l'he passata datatime i es llegeix com a string 
    game_duration = pd.timedelta_range(interruptions['timestamp'][0][-8:], interruptions['timestamp'][len(interruptions)-1][-8:], freq='s')

    # Ara el que farem és anar un per un per cada segon que ha durat el joc i veure si es trobava dins d'una o més interrupcions 
    workload = {}
    for i in game_duration:
        counter = 0
        for j in range(0,len(time_interruption_in)):
            if(time_interruption_in_timedelta[j] <= i <= time_interruption_out_timedelta[j]):
                counter += 1

        workload[str(i)[-8:]] = counter

    return workload




## Workload alternatiu

In [42]:
def calculate_workload_2(row, interruptions):

    workload = 0

    # Check if "piece" column has a value of -1, and if so, add 2 to the workload
    if row['piece'] == -1:
        workload += 2

    # Calculem el temps transcorregut
    # interruptions['timestamp'] = pd.to_datetime(interruptions['timestamp'])
    total_time = (interruptions['timestamp'].max() - interruptions['timestamp'].min()).total_seconds()

    # Calculate the proportion of time elapsed
    timestamp = row['timestamp']
    elapsed_time = (timestamp - interruptions['timestamp'].min()).total_seconds()
    time_proportion = elapsed_time / total_time

    # Add the time proportion to the workload
    workload += time_proportion

    return workload


def trobar_workload_2(interruptions):

    # Primer fem les operacions que necessiten que el timestamp estigui com a string
    timestamp_hms = list(pd.to_datetime(interruptions['timestamp']).dt.strftime("%H:%M:%S"))
    timestamp_timedelta = [ pd.to_timedelta(x) for x in timestamp_hms]
    # Creem un timdelta_range per poder mirar cada segon que dura el joc
    game_duration = pd.timedelta_range(interruptions['timestamp'][0][-8:], interruptions['timestamp'][len(interruptions)-1][-8:], freq='s')

    # Ara passem el timestamp a datetime
    interruptions['timestamp'] = pd.to_datetime(interruptions['timestamp'])

    # Calculem el workload_2 utilitzant la funció calculate_workload_2
    interruptions['workload_2'] = interruptions.apply(lambda row: calculate_workload_2(row, interruptions), axis=1) 

    # Ara el que farem és anar un per un per cada segon que ha durat el joc i veure si es trobava dins d'una o més interrupcions 
    workload_2 = {}
    index = 1
    for i in game_duration:

        if i < timestamp_timedelta[index]:
            workload_2[str(i)[-8:]] = interruptions.iloc[index-1]['workload_2']
        elif i >= timestamp_timedelta[index]:
            workload_2[str(i)[-8:]] = interruptions.iloc[index]['workload_2']
            index += 1

    return workload_2



In [21]:
interruptions = pd.read_csv('Data/csv_interruptions/Aura_with_level1.csv')

# Primer fem les operacions que necessiten que el timestamp estigui com a string
timestamp_hms = list(pd.to_datetime(interruptions['timestamp']).dt.strftime("%H:%M:%S"))
timestamp_timedelta = [ pd.to_timedelta(x) for x in timestamp_hms]
game_duration = pd.timedelta_range(interruptions['timestamp'][0][-8:], interruptions['timestamp'][len(interruptions)-1][-8:], freq='s')

# Ara passem el timestamp a datetime
interruptions['timestamp'] = pd.to_datetime(interruptions['timestamp'])

# Calculem el workload_2 utilitzant la funció calculate_workload_2
interruptions['workload_2'] = interruptions.apply(lambda row: calculate_workload_2(row, interruptions), axis=1) 

# Ara el que farem és anar un per un per cada segon que ha durat el joc i veure si es trobava dins d'una o més interrupcions 
workload_2 = {}
index = 1
for i in game_duration:

    if i < timestamp_timedelta[index]:
        workload_2[str(i)[-8:]] = interruptions.iloc[index-1]['workload_2']
    elif i >= timestamp_timedelta[index]:
        workload_2[str(i)[-8:]] = interruptions.iloc[index]['workload_2']
        index += 1


## Generem el data frame de gazedata amb els dos workloads

In [46]:
def gazedata_definitiu(fitxer):

    """Amb aquesta fucnió retornem el gazedata amb el workload corresponent a cada fila"""

    interruptions = pd.read_csv('Data/csv_interruptions/'+fitxer+'.csv')
    gazedata = pd.read_csv('Data/gazedata_csv/gazedata_'+fitxer+'.csv')

    gazedata['timestamp'] = coordinar_timestamp(gazedata, interruptions, fitxer)

    # Ara escurcem el gazedata al temps en que es juga al joc
    gazedata = gazedata[(pd.to_datetime(interruptions['timestamp'][0]) <= gazedata['timestamp']) &  (gazedata['timestamp'] <= interruptions['timestamp'][len(interruptions)-1])]

    # Resetejem l'índex per despŕes no tenir problemes
    gazedata.reset_index(drop=True, inplace=True)

    """Primer posem el workload que ens diu el nombre d'interrupcions simultànies"""

    workload = trobar_workload(interruptions)

    def assignar_workload(x):

        """Amb aquesta funció trobem per cada instant que hi ha de gazedata quin workload li correspon, workload és un diccionari que s'ha trobat prèviament"""

        #trobem l'espai que separa la data amb l'hora perque no ens interessa el '2023-10-10'
        index_separacio = str(x).index(' ') + 1
        key = str(x)[index_separacio: index_separacio + 8]
        return(workload[key])

    gazedata['workload'] = gazedata['timestamp'].apply(assignar_workload)

    # Com es pot donar algun cas que se solapin mes de 2 interrupcions, classifiquem aquest moment com a workload 2
    gazedata['workload'] = gazedata['workload'].apply(lambda x: 2 if x >= 2 else x)

    print(type(interruptions['timestamp']))

    """Ara calculem el workload_2"""

    workload_2 = trobar_workload_2(interruptions)

    def assignar_workload_2(x):

        """Amb aquesta funció trobem per cada instant que hi ha de gazedata quin workload_2 li correspon, workload_2 és un diccionari que s'ha trobat prèviament"""

        #trobem l'espai que separa la data amb l'hora perque no ens interessa el '2023-10-10'
        index_separacio = str(x).index(' ') + 1
        key = str(x)[index_separacio: index_separacio + 8]
        return(workload_2[key])

    # Workload_2 es per cada segon, però per cada segon tenim 60 mesures, així que amb aquesta funció assignem a cada messura el que li toca
    gazedata['workload_2'] = gazedata['timestamp'].apply(assignar_workload_2)

    # A workload_2 també sumem workload ja que el nombre d'interrupcions simultànies també afecta
    gazedata['workload_2'] = gazedata['workload'] + gazedata['workload_2']

    # Com volem tenir un workload 0, 1 i 2 dividim workload_2 entre 5 i arrodonim
    gazedata['workload_2'] = round(gazedata['workload_2']/2)


    gazedata.to_csv('Data/gazedata_csv_definitiu/'+fitxer+'.csv', index=False)

    return gazedata



In [48]:
for i in df_names:
    gazedata_definitiu(i)

<class 'pandas.core.series.Series'>
16:31:18
<class 'pandas.core.series.Series'>
14:53:52
<class 'pandas.core.series.Series'>
15:06:36
<class 'pandas.core.series.Series'>
15:33:32
<class 'pandas.core.series.Series'>
11:35:58
<class 'pandas.core.series.Series'>
16:33:05
<class 'pandas.core.series.Series'>
16:47:16
<class 'pandas.core.series.Series'>
16:18:36
<class 'pandas.core.series.Series'>
17:26:47
<class 'pandas.core.series.Series'>
12:01:22
<class 'pandas.core.series.Series'>
17:01:20
<class 'pandas.core.series.Series'>
17:12:43
<class 'pandas.core.series.Series'>
17:21:23


In [31]:
a = pd.read_csv('Data/csv_interruptions/Aura_with_level1.csv')

In [45]:
round(2.5)

2